In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import json
import multiprocessing
from seaborn_qqplot import qqplot
from scipy.stats import lognorm, norm
import math
import warnings
warnings.filterwarnings("ignore")

### For the final version of the plots go down. 

In [2]:

df = pd.read_csv("catalogue_v5.csv", header=None)
df.columns = header = ["label", "total_pixels", "x_pixels", "y_pixels",
              "integrated_intensity", "brightest_pixel", "brightest_pixel_x", "brightest_pixel_y",
              "brightest_pixel_RA", "brightest_pixel_DEC", "center_of_mass_x", "center_of_mass_y",
              "center_of_mass_RA", "center_of_mass_DEC", "center_of_gaus_fit_x", "center_of_gaus_fit_y",
              "center_of_gaus_fit_RA", "center_of_gaus_fit_DEC", "fit_x_axis", "fit_y_axis", "fit_theta",
              "deconv_x", "deconv_y", "integrated_intensity_fit", "ratio_residual"
             ]

In [3]:
df.shape

(2291395, 25)

In [ ]:
val1=0
val2=0
for i in list(df.index):
    if i%250000 == 1:
        print(i)
    if(df.deconv_x[i]>0) or (df.deconv_y[i]>0):
        val1 +=1
    if (df.deconv_x[i]>0) or (df.deconv_y[i]>0):
        val2 +=1
print(val1,val2)

1


In [ ]:
sum(df.ratio_residual>10**(-0.7))

Removing NaN's from the data.

In [ ]:
df.dropna(inplace=True)
df.shape

In [ ]:
sum(df.ratio_residual>10**(-0.7))

In [ ]:
val1=0
val2=0
for i in list(df.index):
    if(df.deconv_x[i]>0) or (df.deconv_y[i]>0):
        val1 +=1
    if (df.deconv_x[i]>0) or (df.deconv_y[i]>0):
        val2 +=1
print(val1,val2)

## First lets just check what changes (if any) need to be made for each plot.

In [4]:
df.head()

,label,total_pixels,x_pixels,y_pixels,integrated_intensity,brightest_pixel,brightest_pixel_x,brightest_pixel_y,brightest_pixel_RA,brightest_pixel_DEC,...,center_of_gaus_fit_y,center_of_gaus_fit_RA,center_of_gaus_fit_DEC,fit_x_axis,fit_y_axis,fit_theta,deconv_x,deconv_y,integrated_intensity_fit,ratio_residual
0,P191+55_0,43,7,7,0.961113,0.001262,4195,13,191.526076,53.102602,...,13.168076,191.526239,53.102672,1.719762,1.896624,-0.532125,0.632697,1.986707,0.971715,0.020838
1,P191+55_1,20,7,5,0.326739,0.000757,4605,27,191.241494,53.108014,...,27.736198,191.241573,53.108321,1.519698,1.080085,0.057949,0.000000,0.000000,0.330021,0.193748
2,P191+55_2,45,8,7,0.745168,0.000793,3745,32,191.838427,53.110190,...,32.225611,191.838256,53.110285,2.200380,1.810050,-0.030505,3.293620,1.472263,0.761026,0.051991
3,P191+55_3,43,10,7,0.702810,0.000809,3775,33,191.817606,53.110655,...,33.203413,191.817539,53.110740,1.720535,2.116901,0.055498,0.644249,2.974800,0.705155,0.082425
4,P191+55_4,74,14,9,1.104583,0.000829,4346,35,191.421253,53.111697,...,34.681486,191.421128,53.111564,3.113873,2.061817,-0.772092,6.145499,2.751915,1.080358,0.161090


In [6]:
Filenames = df["label"].str.split("_", n = 1, expand = True) 
df['filename'] =Filenames[0] 

In [ ]:
df.describe()

In [ ]:
#df['label'].groupby(['filename']).agg(['count'])
sns.distplot(df.groupby(['filename']).size())

In [ ]:
sns.distplot(df.groupby(['filename'])[['total_pixels']].sum())

In [ ]:
data_for_hist = ['filename','total_pixels', 'integrated_intensity', 'brightest_pixel', 'fit_x_axis', 'fit_y_axis', 'deconv_x', 'deconv_y', 'integrated_intensity_fit', 'ratio_residual']
data = df[data_for_hist]
#sns.pairplot(data)

In [ ]:
plt.hist(df.total_pixels)
plt.title('Histogram plot of Total Pixels')

plt.xlabel('Total Number Pixels')
plt.ylabel('Counts')

In [ ]:
max(df.total_pixels), min(df.total_pixels)

In [ ]:
np.quantile(df.total_pixels, 0.9999)

In [ ]:
sns.distplot(df.total_pixels)
plt.title('Histogram plot of Total Pixels')

plt.xlabel('Total Number Pixels')
plt.ylabel('Counts')

In [ ]:
## Discarding the last .01% of the largest total pixels value
sns.distplot(df[df.total_pixels<np.quantile(df.total_pixels, 0.9999)].total_pixels)

print(max(df[df.total_pixels<np.quantile(df.total_pixels, 0.9999)].total_pixels))
plt.title('Histogram plot of Total Pixels')

plt.xlabel('Total Number Pixels')
plt.ylabel('Counts')

The cutoff of 99.99% seems quite reasonable. However we can do without this cutoff.

In [ ]:
#df.loc[df.total_pixels == 0, 'total_pixels'] = min(df.loc[df.total_pixels > 0].total_pixels)/1000
sns.distplot(np.log10(df.total_pixels))
plt.title('Histogram plot of Total Pixels LOGSCALE')

plt.xlabel('Total Number Pixels')
plt.ylabel('Counts')

In [ ]:
sns.distplot(df.integrated_intensity)
plt.title('Histogram plot of Integrated Intensity')

plt.xlabel('Integrated Intensity')
plt.ylabel('Counts')


In [ ]:
max(df.integrated_intensity), min(df.integrated_intensity), np.quantile(df.integrated_intensity, 0.99995)

In [ ]:
sns.distplot(df[df.integrated_intensity<np.quantile(df.integrated_intensity, 0.999)].integrated_intensity)
plt.title('Histogram plot of Integrated Intensity')

plt.xlabel('Integrated Intensity')
plt.ylabel('Counts')
## However, a cutoff for this may not be required

In [ ]:
sns.distplot(np.log10(df.integrated_intensity))
plt.title('Histogram plot of Integrated Intensity LOGSCALE')

plt.xlabel('Integrated Intensity')
plt.ylabel('Counts')


In [ ]:
sns.distplot(df.brightest_pixel)
plt.title('Histogram plot of Brightest Pixel')

plt.xlabel('Brightest Pixel')
plt.ylabel('Counts')


In [ ]:
sns.distplot(np.log10(df.brightest_pixel))
plt.title('Histogram plot of Brightest Pixel LOGSCALE')

plt.xlabel('Brightest Pixel')
plt.ylabel('Counts')


For the brightest pixel also, nothing needs to be done.

In [ ]:
sns.distplot(df.fit_x_axis)
plt.title('Histogram plot of Fit X axis')

plt.xlabel('Fit X Axis')
plt.ylabel('Counts')

In [ ]:
df.loc[df.fit_x_axis==0.0, 'fit_x_axis'] =1.0e-8  ## Some values are 0 implying log (0) = inf. So it replaced by a finite value 1000 times less than the minimum value

In [ ]:
sns.distplot(np.log10(df.fit_x_axis))
plt.title('Histogram plot of Fit X axis LOGSCALE')

plt.xlabel('Fit X Axis')
plt.ylabel('Counts')

For the fit_x_axis also, nothing needs to be done.

In [ ]:
sns.distplot(df.fit_y_axis)
plt.title('Histogram plot of Fit Y axis')

plt.xlabel('Fit Y Axis')
plt.ylabel('Counts')

In [ ]:
df.loc[df.fit_y_axis==0.0, 'fit_y_axis'] =1.0e-8 
sns.distplot(np.log10(df.fit_y_axis))
plt.title('Histogram plot of Fit Y axis LOGSCALE')

plt.xlabel('Fit Y Axis')
plt.ylabel('Counts')

For the fit_y_axis also, nothing needs to be done.

In [ ]:
sns.distplot(df.deconv_x)
plt.title('Histogram plot of Deconv X')

plt.xlabel('Deconv X Axis')
plt.ylabel('Counts')

In [ ]:
df.loc[df.deconv_x == 0, 'deconv_x'] = min(df[df.deconv_x > 0].deconv_x)* 1e-5


In [ ]:
sns.distplot(np.log10(df.deconv_x))
plt.title('Histogram plot of Deconv X LOGSCALE')

plt.xlabel('Deconv X Axis')
plt.ylabel('Counts')

In [ ]:
sns.distplot(df.deconv_y)
plt.title('Histogram plot of Deconv y')

plt.xlabel('Deconv y Axis')
plt.ylabel('Counts')

In [ ]:
df.loc[df.deconv_y == 0, 'deconv_y'] = min(df[df.deconv_y > 0].deconv_y)* 1e-5
sns.distplot(np.log10(df.deconv_y))
plt.title('Histogram plot of Deconv y LOGSCALE')

plt.xlabel('Deconv y Axis')
plt.ylabel('Counts')

In [ ]:
sns.distplot(df.integrated_intensity_fit)
plt.title('Histogram plot of Integrated Intensity')
plt.xlabel('Integrated Intensity Fit ')
plt.ylabel('Counts')

In [ ]:
max(df.integrated_intensity_fit),min(df.integrated_intensity_fit),np.quantile(df.integrated_intensity_fit,0.99)

The minimum is negetive which will cause problems. So we may try shifting it by the minimum amount. However this will render the value which is minimum to zero. So lets shift is by the minimum amount + the difference between the smallest and the second smallest values.

In [ ]:
smallest, second_smallest = min(df.integrated_intensity_fit), min(df[df.integrated_intensity_fit> min(df.integrated_intensity_fit)].integrated_intensity_fit)
smallest, second_smallest

In [ ]:
shift = abs(smallest)+abs(smallest - second_smallest)
shift

In [ ]:
#df.loc[df.integrated_intensity_fit == 0, 'integrated_intensity_fit'] = min(df.loc[df.integrated_intensity_fit > 0].integrated_intensity_fit)/1000
sns.distplot(np.log10(df.integrated_intensity_fit+ shift))
plt.title('Histogram plot of Integrated Intensity Fit LOGSCALE')
plt.xlabel('Integrated Intensity Fit ')
plt.ylabel('Counts')

Also, another way to deal with this is to simply remove the ones with negetive intensity fit

In [ ]:
sns.distplot(np.log10(df[df.integrated_intensity_fit>0].integrated_intensity_fit))
plt.title('Histogram plot of Integrated Intensity Fit LOGSCALE')
plt.xlabel('Integrated Intensity Fit ')
plt.ylabel('Counts')

In [ ]:
sns.distplot(df.ratio_residual)
plt.title('Histogram plot of Ratio Residual')

plt.xlabel('Ratio Residual')
plt.ylabel('Counts')

In [ ]:
df.loc[df.ratio_residual == 0, 'ratio_residual'] = min(df.loc[df.ratio_residual > 0].ratio_residual)*1e-5

sns.distplot(np.log10(df.ratio_residual))
plt.title('Histogram plot of Ratio Residual LOGSCALE')

plt.xlabel('Ratio Residual')
plt.ylabel('Counts')

For the Ratio Residual nothing needs to be done.

In [ ]:
data.head()

## Final Plots:

<a id='Final Plots'></a>

Cleaning the data according to what we just saw:
1. Remove total pixels which have values greater than 99.99% of the data ie. cutoff = 6339 - $\textit{this is ommited for now}$
2. Shift the integrated Intensity fit values by the minimum+difference between 
3. For the deconv_x and deconv_y we replace the 0 values with the minimum multiplied by 1e-5
4. Also for the Ratio Residuals, the minimum value 0 is to be replaced by the minimum(non -zero) multiplied by 1e-5

In [ ]:
#data = data[data.total_pixels<np.quantile(data.total_pixels, 0.9999)]
data.integrated_intensity_fit += shift
data.loc[data.deconv_x == 0, 'deconv_x'] = min(data[data.deconv_x > 0].deconv_x)* 1e-5
data.loc[data.deconv_y == 0, 'deconv_y'] = min(data[data.deconv_y > 0].deconv_y)* 1e-5
data.loc[data.ratio_residual == 0, 'ratio_residual'] = min(data.loc[data.ratio_residual > 0].ratio_residual)*1e-5
data.loc[data.fit_x_axis == 0, 'fit_x_axis'] = min(data[data.fit_x_axis > 0].fit_x_axis)* 1e-5
data.loc[data.fit_y_axis == 0, 'fit_y_axis'] = min(data[data.fit_y_axis > 0].fit_y_axis)* 1e-5


In [ ]:
def fairplot(x, xlab="x", ylab="y", main= "Histogram", bins1=None, bins2=None, hist=[True,True], kde=[False,False], 
             fit1=None, fit2 = None, rug=[False,False], color1=None, color2 =None, vertical=[False,False], 
             label1=None, label2=None, norm_hist=[False,False], dpi= 100, figsize = (7, 3.5), label_size=(20,18),
             xlim1=None, xlim2=None, ylim1=None, ylim2=None, yscale = ['linear', 'linear'], scale_base=None):
    
    f, (ax1,ax2) = plt.subplots(1, 2, figsize=figsize, sharex=False, dpi =dpi)
    sns.distplot(x, bins=bins1, hist=hist[0], kde=kde[0], fit=fit1, rug=rug[0], color=color1, vertical=vertical[0], 
                 label=label1,norm_hist=norm_hist[0], ax=ax1)
    sns.distplot(np.log10(x), bins=bins2,  hist=hist[1], kde=kde[1], fit=fit2, rug=rug[1], color=color2, vertical=vertical[1], 
                 label=label2, norm_hist=norm_hist[1], ax=ax2)
    ax1.set_xlabel(xlab, fontsize=label_size[1])
    ax1.set_ylabel(ylab, fontsize=label_size[1])
    ax1.set_title('Histogram', pad=7, fontsize = label_size[0])
    if xlim1 != None:
        ax1.set(xlim=xlim1)
    if ylim1 !=None:
        ax1.set(ylim=ylim1)
    ax2.set_xlabel('Log $_{10}$'+xlab, fontsize = label_size[1])
    ax2.set_ylabel(ylab,fontsize = label_size[1])
    ax2.set_title('Histogram in LogScale', pad=7, fontsize = label_size[0])
    if xlim2 != None:
        ax2.set(xlim=xlim2)
    if ylim1 !=None:
        ax2.set(ylim=ylim2)
    if yscale[0]=='log':
        ax1.set_yscale(yscale[0], basey=scale_base)
    else:
        ax1.set_yscale(yscale[0])
    if yscale[1]=='log':
        ax2.set_yscale(yscale[1], basey=scale_base)
    else:
        ax2.set_yscale(yscale[1])
    f.suptitle(main, fontsize=12, y=1.05)
    f.tight_layout() 
    
    return(f)

In [ ]:
dpi_set =600
ls = (18,14)

In [ ]:
fig_intgrtdInt = fairplot(data.integrated_intensity, main = "Plot of Integrated Intensity",xlab = "Integrated Intensity", 
                          ylab = "Counts", kde=[False,False],dpi=dpi_set, label_size = ls)

plt.savefig('IntegratedIntensity.png')

In [ ]:
fig_intgrtdIntLog = fairplot(data.integrated_intensity, main = "Plot of Integrated Intensity",xlab = "Integrated Intensity", 
                          ylab = "Counts", kde=[False,False],dpi=dpi_set, yscale = ['log','log'], scale_base = 10,
                             label_size = ls)

plt.savefig('IntegratedIntensity_Log.png')

In [ ]:
fig_IntgrtdIntFit = fairplot(data.integrated_intensity_fit, main = "Plot of the fit of Integrated Intensity", 
                             xlab = "Integrated Intensity fit", ylab = "Counts", kde=[False,False], dpi=dpi_set,
                             label_size = ls)

plt.savefig('IntegratedIntensityFit.png')

In [ ]:
fig_IntgrtdIntFitLog = fairplot(data.integrated_intensity_fit, main = "Plot of the fit of Integrated Intensity", 
                             xlab = "Integrated Intensity fit", ylab = "Counts", kde=[False,False], dpi=dpi_set,
                            yscale = ['log','log'], scale_base = 10, label_size = ls)

plt.savefig('IntegratedIntensityFit_Log.png')

In [ ]:
fig_totpxls = fairplot(data.total_pixels, main = "Plot of Total Pixel area of source",xlab = "Total Pixels", ylab = "Counts", 
                     kde=[False,False],dpi=dpi_set, label_size = ls)
plt.savefig('TotalPixels.png')

In [ ]:
fig_totpxlsLog = fairplot(data.total_pixels, main = "Plot of Total Pixel area of source",xlab = "Total Pixels", ylab = "Counts", 
                     kde=[False,False],dpi=dpi_set,  yscale = ['log','log'], scale_base = 10, label_size = ls)
plt.savefig('TotalPixels_Log.png')

In [ ]:
fig_brightestpxl = fairplot(data.brightest_pixel, main = "Plot of Brightest Pixel",xlab = "Brightest Pixel Intensity", ylab = "Counts", 
                     kde=[False,False],dpi=dpi_set, label_size = ls)
plt.savefig('BrightestPixels.png')

In [ ]:
## Changed the y axis for the plots to Log scale
fig_brightestpxlLog = fairplot(data.brightest_pixel, main = "Plot of Brightest Pixel",xlab = "Brightest Pixel Intensity", ylab = "Counts", 
                     kde=[False,False],dpi=dpi_set, yscale = ['log','log'], scale_base = 10, label_size = ls)
plt.savefig('BrightestPixels_Log.png')

In [ ]:
fig_xaxis = fairplot(data.fit_x_axis, main = "Plot of X axis fit",xlab = "Fit x axis", ylab = "Counts", 
                     kde=[False,False],dpi=dpi_set, yscale=['linear','linear'], label_size = ls)

plt.savefig('Fit_Xaxis.png')

In [ ]:
## Changed the y axis for the plots to Log scale
fig_xaxisLog = fairplot(data.fit_x_axis, main = "Plot of X axis fit",xlab = "Fit x axis", ylab = "Counts", 
                     kde=[False,False],dpi=dpi_set, yscale = ['log','log'], scale_base = 10, label_size = ls)
plt.savefig('Fit_Xaxis_Log.png')

In [ ]:
fig_yaxis = fairplot(data.fit_y_axis, main = "Plot of Y axis fit", 
                             xlab = "Fit y axis", ylab = "Counts", dpi=dpi_set, kde=[False,False], label_size = ls)
plt.savefig('Fit_Yaxis.png')

In [ ]:
fig_yaxisLog = fairplot(data.fit_y_axis, main = "Plot of Y axis fit", kde=[False,False], label_size = ls,
                             xlab = "Fit y axis", ylab = "Counts", dpi=dpi_set, yscale = ['log','log'], scale_base = 10)
plt.savefig('Fit_Yaxis_Log.png')

In [ ]:
fig_Deconvx = fairplot(data.deconv_x, main = "Plot of Deconv x", kde=[False,False], label_size = ls,
                             xlab = "Deconv x", ylab = "Counts", dpi=dpi_set)
plt.savefig('Deconv_X.png')

In [ ]:
fig_DeconvxLog = fairplot(data.deconv_x, main = "Plot of Deconv x", kde=[False,False], label_size = ls,
                             xlab = "Deconv x", ylab = "Counts", dpi=dpi_set, yscale = ['log','log'], scale_base = 10)
plt.savefig('Deconv_X_Log.png')

In [ ]:
fig_Deconvy = fairplot(data.deconv_y, main = "Plot of Deconv y", kde=[False,False], label_size = ls,
                             xlab = "Deconv y", ylab = "Counts", dpi=dpi_set)
plt.savefig('Deconv_Y.png')

In [ ]:
fig_DeconvyLog = fairplot(data.deconv_y, main = "Plot of Deconv y", kde=[False,False], label_size = ls,
                             xlab = "Deconv y", ylab = "Counts", dpi=dpi_set, yscale = ['log','log'], scale_base = 10)
plt.savefig('Deconv_Y_Log.png')

In [ ]:
fig_RatioResidual = fairplot(data.ratio_residual, main = "Plot of Ratio Residuals", xlab = "Ratio Residual", ylab = "Counts", 
                       kde=[False,False], dpi=dpi_set, label_size = ls, bins1 = 100, bins2 = 100)
plt.savefig('RatioResidual.png')

In [ ]:
fig_RatioResidualLog = fairplot(data.ratio_residual, main = "Plot of Ratio Residuals", xlab = "Ratio Residual", ylab = "Counts", 
                       kde=[False,False], dpi=dpi_set, yscale = ['log','log'], scale_base = 10, label_size = ls)
plt.savefig('RatioResidual_Log.png')

In [ ]:
## Fitting the Ratio Residuals plot with a lognormal function and that of the log with a normal function.
fig_RatioResidualfit = fairplot(data.ratio_residual, main = "Plot of Ratio Residuals", xlab = "Ratio Residual", ylab = "Density", 
                     kde= [False,False], fit1 =lognorm, fit2=norm, dpi=dpi_set, label_size = ls)
plt.savefig('RatioResidual_Fit_Lognorm_norm.png')

In [ ]:
fig_Files = fairplot(data.groupby(['filename']).size(), main = "Plot of number of sources per file", 
                     xlab = "Number of sources/file", ylab = "Counts", kde=[False,False], dpi=dpi_set, label_size = ls)
plt.savefig('Sourcesperfile.png')

In [ ]:
fig_Files = fairplot(data.groupby(['filename'])[['total_pixels']].sum(), main = "Plot of total pixels from all files  per file", 
                     xlab = "Total pixels from all sources/file", ylab = "Counts", kde=[False,False], dpi=dpi_set,
                     label_size = (16,10))
plt.savefig('TotalPixelsperfile.png')


In [ ]:
sns.set(font_scale=1.6) 
sns_plot = sns.pairplot(data[['total_pixels','integrated_intensity','integrated_intensity_fit', 'ratio_residual']], size = 3.5)
sns_plot.savefig("Pairplot.png")

In [ ]:
data.describe()

In [ ]:
sns.set(font_scale=1.6) 
sns_plot = sns.pairplot(data[['fit_x_axis','fit_y_axis','deconv_x', 'deconv_y','ratio_residual']], size = 3.5)
sns_plot.savefig("Pairplot_2.png")

In [ ]:
#sns.set(font_scale=1.6) 
#sns_plot = sns.pairplot(data)
#sns_plot.savefig("Pairplot_all.png")

In [ ]:
#sns_plot = sns.pairplot(data, hue = "filename")
#sns_plot.savefig("Pairplot_all_filecolored.png")

In [ ]:
corr = data.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sns.set(font_scale=1.6) 
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(210, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.xticks(rotation=60, ha='right')
plt.tight_layout()
f.savefig("Correlation_data.png")

In [ ]:
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sns.set(font_scale=1.6) 
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(210, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .75})
plt.xticks(rotation=60, ha='right')
plt.tight_layout()
f.savefig("Correlation_data_all.png")

In [ ]:
f, ax = plt.subplots(figsize =(8,5))
ax.scatter(df.integrated_intensity_fit- df.integrated_intensity, df.ratio_residual)

ax.set_xlabel("Difference : Integrated Intensity fit - Integrated Intensity")
ax.set_ylabel("Ratio residual")
plt.tight_layout()
f.savefig("Difference.png")

In [ ]:
fig, ax = plt.subplots()
#sns.distplot(data.integrated_intensity, ax = ax, bins = 10)
#sns.distplot(data.integrated_intensity_fit - shift, ax = ax, bins=10)
#ax.set_xscale('log', basex=10)
#ax.set_yscale('log', basey=10)
#sns.pairplot(data, vars= ['integrated_intensity_fit', 'integrated_intensity'] )
ax.scatter(data.integrated_intensity_fit - shift, data.integrated_intensity)

In [ ]:
df_ = pd.read_csv('MOSAICs_DICT.csv')
azm =df[['total_pixels','integrated_intensity', 'integrated_intensity_fit', 'ratio_residual', 'filename']]
azm = azm.groupby(['filename']).sum().reset_index()
azm = (pd.merge(azm, df_, left_on='filename', right_on='File'))
azm = azm.drop(columns= ['File', 'Unnamed: 0'])
azm['Background'] = (azm.Counts_NonZero-azm.total_pixels)
azm['Threshold'] = (azm.Counts_NonZero/azm.total_pixels)

In [ ]:
f, ax = plt.subplots()
sns.distplot(np.log10(azm.Background), kde=False, ax=ax)

ax.set_xlabel("Difference : Nonzero_values(Mosaic) - total_pixels")
ax.set_ylabel("Counts")
#ax.set_xscale('log', basex=10)
#plt.tight_layout()
f.savefig("Background.png")

In [ ]:
f, ax = plt.subplots()
sns.distplot(np.log10(azm.Threshold), kde=False, ax=ax)

ax.set_xlabel("Difference : Nonzero_values(Mosaic)/total_pixels")
ax.set_ylabel("Counts")
#ax.set_xscale('log', basex=10)
#plt.tight_layout()
f.savefig("NoiseThreshold.png")

In [ ]:
azm.loc[azm['Background'].idxmax()]

In [ ]:
azm.nlargest(3, 'Background')

In [5]:
qqplot(data, x="ratio_residuals", y=norm, height = 4, aspect = 1.5, display_kws={"identity":False,"fit":True,"reg":True,"ci":0.025})

NameError: name 'data' is not defined

In [ ]:
from scipy.spatial.distance import cdist
dummy_array = azm[["CrvalX", "CrvalY"]].to_numpy()
dist_mat = cdist(dummy_array, dummy_array)
distances = pd.DataFrame(dist_mat, columns = azm["filename"], index = azm["filename"])

In [ ]:
distances

In [ ]:
df_ = pd.read_csv("overlap_dict.csv")
df_